## Loading data

In [ ]:
from sparql import DB
virtuoso = DB(triplestore="virtuoso", protocol="http",url="localhost",port="8890")

In [ ]:
import json
import requests
from requests.auth import HTTPDigestAuth
from rdflib import Graph

In [ ]:
virtuoso.sparql_endpoint

In [ ]:
!ls data

In [ ]:
g = Graph()

In [ ]:
g.parse("data/example-data.ttl", format="ttl")

In [ ]:
#g.serialize()

## Use SPARQL Update

In [ ]:
# this works
from SPARQLWrapper import SPARQLWrapper
queryString = "INSERT DATA { GRAPH <https://golemlab.eu/data> { <https://golemlab.eu/data/1> a <https://golemlab.eu/data/corpus> . } }" 
sparql = SPARQLWrapper("http://localhost:8890/sparql-auth")
sparql.setHTTPAuth("DIGEST")
sparql.setCredentials("dba", "pwd123")
sparql.setQuery(queryString) 
sparql.method = 'POST' 
sparql.query()

documentation https://vos.openlinksw.com/owiki/wiki/VOS/VirtRDFInsert

## Upload a ttl file

this works:
```
curl --digest --user dba:pwd123 --url "http://localhost:8890/sparql-graph-crud-auth?graph=https://golemlab.eu/data" -X POST -T example-data.ttl
```

In [ ]:
ttl_content = g.serialize(format="ttl")

In [ ]:
graph_uri = "https://golemlab.eu/data"
url =  "http://localhost:8890/sparql-graph-crud-auth" + "?graph=" + graph_uri
print(url)
data = ttl_content.encode(encoding='utf-8')
response = requests.post(url=url, data=data, auth=HTTPDigestAuth("dba", "pwd123"), headers={'Content-Type': 'application/x-turtle'})
print(response.status_code)
print(response.text)